In [28]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import median_filter
from sklearn.model_selection import train_test_split
import warnings
import logging
from tqdm import tqdm
from keras.layers import Input, Conv1D, Conv2D, MaxPooling2D, concatenate, Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Conv1D, Activation, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

seed = 42
training_labels = {
    "Wish": 0,
    "Another": 1,
    "Comfortably": 2,
    "Money": 3,
    "Breathe": 4,
    "Time": 5,
    "Brain": 6,
    "Echoes": 7,
    "Wearing": 8,
    "Sorrow": 9,
    "Hey": 10,
    "Shine": 11,
}
data = np.load("../../data/x_train.npy")
target = np.load("../../data/y_train.npy")
#SHUFFLE DATA
p = np.random.permutation(data.shape[0])
data = data[p]
target = target[p]

test_split = 0.2
val_split = 0.1
#OneHotEncoding
target = tfk.utils.to_categorical(target)

#Split Train Test
x_train,x_test,y_train,y_test = train_test_split(data,target, test_size=test_split,random_state=42,stratify=target)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train, test_size=val_split,random_state=42,stratify=y_train)
x_train.shape,x_test.shape,x_val.shape


def normalize_single_series(data):
  for i in tqdm(range(0,data.shape[0])):
    chunck = data[i]
    mean = np.mean(chunck)
    std  = np.std(chunck)
    data[i] = (chunck - mean) / std
  return data


x_old = x_train.copy()
x_train = normalize_single_series(x_train)
x_test  = normalize_single_series(x_test)
x_val   = normalize_single_series(x_val)

def add_noise(seq,weight=0.005):
  #Copy Data
  seq = np.array(seq)
  seq = seq.copy()
  #Calculate STD
  std = np.std(seq) * weight
  #Calculate Mean of this sequence
  noise = np.random.normal(0,std,seq.shape)

  data_augmented = seq + noise*0.1
  return data_augmented

augmentation = False #NOW IS FALSE SINCE IS TO CORRECT THE WAY IT ADD NOISE
np.random.seed(45)

y_new = y_train.copy()
x_new = x_train.copy()
if augmentation:
  weights = [0.01,0.02,0.05]
  for w in weights:
    for i in tqdm(range(0,x_train.shape[0])):
      aug = add_noise(x_train[i],w)
      aug = np.expand_dims(aug, axis=0)
      #print(aug.shape,x_new.shape)
      #print(y_new.shape,y_train.shape)
      x_new    = np.vstack([x_new,aug])
      y_new    = np.vstack([y_new,y_train[i]])

x_train = x_new
y_train = y_new
x_train.shape,y_train.shape

2.10.0


100%|██████████| 195/195 [00:00<00:00, 53064.90it/s]


((1748, 36, 6), (1748, 12))

In [ ]:
# First, we'll define the structure of the autoencoder.
# This example uses a simple autoencoder with one hidden layer
# that has a reduced dimension of 3.
input_dim = 6
timesteps = 36
encoding_dim = 3

# define the input layer
input_data = Input(shape=(timesteps, input_dim))


# define the first hidden layer
encoded = tfkl.LSTM(64, return_sequences=True)(input_data)
encoded = tfkl.LSTM(32, return_sequences=True)(encoded)
encoded = tfkl.LSTM(16, return_sequences=True)(encoded)
encoded = tfkl.LSTM(8)(encoded)

# define the decoder
decoded = tfkl.RepeatVector(timesteps)(encoded)
decoded = tfkl.LSTM(8, return_sequences=True)(decoded)
decoded = tfkl.LSTM(16, return_sequences=True)(decoded)
decoded = tfkl.LSTM(32, return_sequences=True)(decoded)
decoded = tfkl.LSTM(64, return_sequences=True)(decoded)

# define the output layer
decoded = tfkl.TimeDistributed(Dense(input_dim))(decoded)

# define the model
autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics="accuracy")

# Now we'll train the autoencoder on the data.
# The data should be in the form of a 3D array with dimensions (num_samples, timesteps, input_dim)
# You can use the 'fit' method to train the autoencoder.

# Here's an example:
autoencoder.fit(x_train, x_train, epochs=100, batch_size=32, shuffle=True, validation_data=(x_val, x_val))

# Once the autoencoder is trained, we can use it to augment the data by encoding and decoding the data.
# This will add additional variation to the data, which can be useful for training a model.

# Here's how to encode and decode a dataset using the autoencoder:
encoded_data = autoencoder.predict(data)
decoded_data = autoencoder.predict(encoded_data)

# The encoded data will have the same number of samples as the original data, but with a reduced dimension.
# The decoded data will have the same dimension as the original data.
# You can use the encoded and decoded data as additional training samples for your model.

Epoch 1/100


2022-12-15 16:07:40.829532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.354673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.504184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.623665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.740248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.858133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:41.982245: I tensorflow/core/grappler/optimizers/cust

55/55 [==============================] - ETA: 0s - loss: 0.9546 - accuracy: 0.2182

2022-12-15 16:07:54.455837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:54.638131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:54.747043: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:54.849964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:54.956778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:55.072691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:07:55.185846: I tensorflow/core/grappler/optimizers/cust

55/55 [==============================] - 21s 226ms/step - loss: 0.9546 - accuracy: 0.2182 - val_loss: 0.9077 - val_accuracy: 0.2131
Epoch 2/100
55/55 [==============================] - 6s 111ms/step - loss: 0.8771 - accuracy: 0.2069 - val_loss: 0.8274 - val_accuracy: 0.2041
Epoch 3/100
55/55 [==============================] - 6s 105ms/step - loss: 0.8117 - accuracy: 0.2221 - val_loss: 0.7894 - val_accuracy: 0.2007
Epoch 4/100
55/55 [==============================] - 6s 104ms/step - loss: 0.7897 - accuracy: 0.2219 - val_loss: 0.7619 - val_accuracy: 0.2372
Epoch 5/100
55/55 [==============================] - 6s 102ms/step - loss: 0.7697 - accuracy: 0.2261 - val_loss: 0.7504 - val_accuracy: 0.2098
Epoch 6/100
55/55 [==============================] - 6s 103ms/step - loss: 0.7592 - accuracy: 0.2207 - val_loss: 0.7423 - val_accuracy: 0.2291
Epoch 7/100
55/55 [==============================] - 6s 108ms/step - loss: 0.7459 - accuracy: 0.2169 - val_loss: 0.7394 - val_accuracy: 0.2345
Epoch 8/10